In [ ]:
%matplotlib inline
#這是jupyter notebook的magic word˙

import matplotlib
import matplotlib.pyplot as plt
from IPython import display

# iWildCam2020 資料預處理

In [ ]:
import os
#判斷是否在jupyter notebook上
def is_in_ipython():
    "Is the code running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))

    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'jupyter' in program_name or  # jupyter
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False


#判斷是否在colab上
def is_in_colab():
    if not is_in_ipython(): return False
    try:
        from google import colab
        return True
    except: return False

#判斷是否在kaggke_kernal上
def is_in_kaggle_kernal():
    if 'kaggle' in os.environ['PYTHONPATH']:
        return True
    else:
        return False

if is_in_colab():
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
os.environ['TRIDENT_BACKEND'] = 'pytorch'

if is_in_kaggle_kernal():
    os.environ['TRIDENT_HOME'] = './trident'
    
elif is_in_colab():
    os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'

#為確保安裝最新版 
!pip uninstall tridentx -y
!pip install ../input/trident/tridentx-0.7.5-py3-none-any.whl --upgrade
import json
import copy
import numpy as np
#調用trident api
import trident as T
from trident import *
from trident.models import resnet,efficientnet
import cv2
import glob
import json

由於這個競賽原始數據非常巨大，也因此我們無法直接載入，我已經將所需要的部分檔案上傳到我的google drive並開放權限。然後您只需要利用trident api中的download_file_from_google_drive函數即可方便的下載，預設下載目錄會是trident api的主資料夾下方的downloads資料夾。下載後，我們可以透過讀讀取標註json檔來解讀標註內容。

Python是一種動態型別的語言，他不像C#這類語言需要事先宣告型別，很方便但是也很惱人，因為它不是真的沒有型別，所以你如果做的操作與型別不符仍會報錯，所以介紹一下我個人在分析時確認型別的訣竅，我會先列印出該物件的\_\_class\_\_\.\_\_mro\_\_，也就是型別繼承的順序，例如還原過後的標註檔，它的mro顯示它是繼承dict，所以我們就可以進一步檢視它的keys()，若它是個清單，我們則可以列印出它的前5筆內容，這是我個人在處以未知型別時常用的手法。雖然\_\_class\_\_\.\_\_name\_\_就能檢視型別名稱，但是萬一這個型別我們不認識，或者是它是繼承多種型別，那就還是無法處理，所以我乾脆使用mro，可以顯示整個物件繼承順序，這樣在實務上會比較方便判讀。

In [ ]:

with open('../input/iwildcam-2020-fgvc7/iwildcam2020_train_annotations.json') as json_file:
    train_data = json.load(json_file)
    #先查看文件還原後的型別
    print(train_data.__class__.__mro__)
    print(train_data.__class__.__name__)
    #確認是dict，就先檢查keys
    print(train_data.keys())
    #查看各個keys型別
    print(train_data['annotations'].__class__.__mro__)
    print(train_data['images'].__class__.__mro__)
    print(train_data['categories'].__class__.__mro__)
    print(train_data['info'].__class__.__mro__)
    #查看各個keys內容
    print('annotations內容')
    print(train_data['annotations'][:5])
    print('images內容')
    print(train_data['images'][:5])
    print('categories內容')
    print(train_data['categories'][:5])
    print('info內容')
    print(train_data['info'].keys())

with open('../input/iwildcam-2020-fgvc7/iwildcam2020_test_information.json') as test_json_file:
    test_data = json.load(test_json_file)
    #先查看文件還原後的型別
    print('以下為測試集數據')
    print(test_data.__class__.__mro__)
    print(test_data.keys())
    print('images內容')
    print(train_data['images'][:5])
    print('categories內容')
    print(train_data['categories'][:5])
   

了解你的數據是在做數據分析前非常重要的階段，我們也稱這個階段為DAE (Data Exploration Analysis)，如果是處理表格型態的內容，最方便的莫過於pandas。身為數據科學家，千萬不要講到熊貓直覺反應是要點外賣。pandas最大的好處在於它整合了多種數據來源的讀取與寫入，即使是像這次屬於非結構數據的json檔，它也能轉換成表格型態數據(DataFrame)

In [ ]:
import pandas as pd
df_train = pd.DataFrame({'id': [item['id'] for item in train_data['annotations']],
                         'category_id': [item['category_id'] for item in train_data['annotations']],
                         'image_id': [item['image_id'] for item in train_data['annotations']],
                         'location': [item['location'] for item in train_data['images']],
                         'file_name': [item['file_name'] for item in train_data['images']]})
df_test = pd.DataFrame({'image_id': [item['id'] for item in train_data['images']],
                         'location': [item['location'] for item in train_data['images']],
                         'file_name': [item['file_name'] for item in train_data['images']]})



df_train

接下來可以透過類似的方式將train_data['categories']取出建構成新的dtaframe，並列印一下，這才知道，原來出現最多次的是「人類」這種動物。而除了人類這種動物之外，是一種叫做「眼斑吐綬雞(meleagris ocellata)」的動物。而更變態的是，標註中的類別還出現了0次的動物，而且數量還不少。所以我們這時可以做個過濾，排除掉不存在於圖片標註中的類別

In [ ]:
df_category_train=pd.DataFrame({'id': [item['id'] for item in train_data['categories']],
                         'name': [item['name'] for item in train_data['categories']],
                         'count': [item['count'] for item in train_data['categories']]})

df_category_test=pd.DataFrame({'id': [item['id'] for item in test_data['categories']],
                         'name': [item['name'] for item in test_data['categories']],
                         'count': [item['count'] for item in test_data['categories']]})

df_category_train=df_category_train.sort_values(['count'],ascending=False) 
print(df_category_train)
df_category_test=df_category_test.sort_values(['count'],ascending=False) 
print(df_category_test)

我們可以利用is_in這個函數來進行欄位的篩選，這才知道像是舊世界綠猴(chlorocebus pygerythrus)等動物其實只出現過一次，而原本類別檔中出現最多次的人類竟然不見了，可見category中的數量是完全不可信的，只能參考它的category_id以及名稱之間的對應。更糟的是，雖然訓練及與測試集的類別表看起來是一致的，但是由於它不可能給測試集的標註，我們只能根據category中數量大於零來作為判斷，然後一去重複比對之下，竟然出現測試集出現了意料之外的動物的這種劇情。我檢查半天我沒寫錯，查了一下討論區，看到了以下留言：

https://www.kaggle.com/c/iwildcam-2020-fgvc7/discussion/143071

好吧，只能暫時相信出題方了。而也因此我開始擔心圖片annotations中的註記是否跟圖片一致....

In [ ]:
#基於標註檔，產生有在標註檔內的所有圖片的清單，進行去重複(set)、排序(sorted)以及轉換成清單(list)
animal_category_lists=list(sorted(set([item['category_id'] for item in train_data['annotations']])))
print(len(animal_category_lists))

#進行篩選
df_category_train=df_category_train[df_category_train['id'].isin(animal_category_lists)]
print(len(df_category_train))

df_category_test=df_category_test[df_category_test['count']>0]
print(len(df_category_test))


animal_category_lists_train=[category_id.item() for category_id in df_category_train[['id']].to_numpy().astype(np.int64)]
print(animal_category_lists_train[:5])

animal_category_lists_test=[category_id.item() for category_id in df_category_test[['id']].to_numpy().astype(np.int64)]
print(animal_category_lists_test[:5])

#檢查是不是所有df_category_test數量不為零的動物都有出現在df_category_train的類別代號中
category_missing_list=[category_id for category_id in animal_category_lists_test if category_id not in animal_category_lists_train]
print(category_missing_list)









除了數量外，我們也要注意到，出現最多的動物以及出現最少的動物，其圖片數量相差了1.6萬倍，我們可以透過value_counts()來計算次數分配表，然後透過pandas中的plot自動畫圖。

In [ ]:
#設定輸出圖的尺寸與dpi
fig=plt.figure(figsize=(12, 6), dpi=96)
#列印出每種動物的照片數頻率分布(一張照片只計算為1，不管裡面出現多少隻)
df_animal_frequency=df_train['category_id'].value_counts()
#列印df_animal_frequency直接會自動展示頭尾5筆以及欄位定義與型別
print(df_animal_frequency)
#而且還可以自動畫圖
df_animal_frequency.plot(kind='bar')

我們可以透過以下語法抓出所有圖片，以及統計一下數量。經過去重複，還好去重前後數量都是217959，這個數字是合的。沒有出現我覺得最討人厭的一張照片出現兩種動物，也因此，這個題目是可以符合圖像識別的要求(但這只是最低要求，我只是要用它做個baseline以及展示一下多類別不均衡如何解)，那我們就開始來做出baseline吧。

In [ ]:
import glob
#透過glob所全部train資料夾中所有可用圖片
imgs=glob.glob('../input/iwildcam-2020-fgvc7/train/*.jpg')
print(len(imgs))
print(imgs[:5])

#將圖檔路徑去除資料夾部分後進行去重複
img_pathes=[img.split('/')[-1] for img in imgs]
img_pathes=list(sorted(set(img_pathes)))
print(len(img_pathes))

print(img_pathes[:5])

但是仔細看可以發現，我們的動物類別id並不是連續數字，中間是有跳過的，而且既然出題方都說了測試集不會出現意料之外的動物，那麼我們就按照訓練集出來的類別清單(animal_category_lists_train)來建構標籤囉。

In [ ]:
label2category=OrderedDict()
category2label=OrderedDict()
#產生能將category_id轉label的字典
for i in range(len(animal_category_lists_train)):
    category2label[animal_category_lists_train[i]]=i
    label2category[i]=animal_category_lists_train[i]

#建構出轉成標籤id
label_idxes=[category2label[item['category_id']] for item in train_data['annotations']]
image_pathes=['../input/iwildcam-2020-fgvc7/train/'+item['file_name'] for item in train_data['images']]

print(label_idxes[:5])
print(image_pathes[:5])

In [ ]:

img_ds=ImageDataset(image_pathes,symbol='image')
label_ds=LabelDataset(label_idxes,symbol='label')

#與Iterator構成data provider
data_provider=DataProvider(traindata=Iterator(data=img_ds,label=label_ds))

data_provider.image_transform_funcs=[
    RandomAdjustGamma(scale=(0.6,1.4)),#調整明暗
    RandomAdjustHue(scale=(-0.5,0.5)),#調整色相
    RandomAdjustSaturation(scale=(0.6,1.4)),#調整飽和度
    SaltPepperNoise(0.01),#加入胡椒鹽噪音
    RandomErasing(), #加入隨機擦去
    RandomRescaleCrop((224,224),scale=(0.5,2)),#縮放裁切至(224,224)
    Normalize(127.5,127.5)] #標準化

data,labels=data_provider.next()
print(data.shape)
print(labels)



用preview_images預覽一下圖片，一看嚇到，是我運氣不好嗎?一預覽就看到一堆純黑的圖。

In [ ]:
%%time
data_provider.preview_images()

我決定先取200筆圖片(不使用data_provider，畢竟上面加入了數據增強，所以其輸出圖片不等於真實圖片)檢查到底那些圖是真的全黑還是接近黑而已，以及它們對應的標籤是不是0(無動物)

In [ ]:
image_samples=image_pathes[:200]
label_samples=label_idxes[:200]
image_avg_pixel=[]
for i in range(200):
    arr=image2array(image_samples[i])
    avg_pixel=arr.mean()
    image_avg_pixel.append(avg_pixel)
    if avg_pixel<30:
        print('label:{0} image_path:{1} avg_pixel:{2}  shape:{3}'.format(label_samples[i],image_samples[i],avg_pixel,arr.shape),flush=True)
        display.display(array2image(image2array(image_samples[i])))

還好只是自己嚇自己，原來是用錯了縮放方法，RandomRescaleCrop((224,224),scale=(0.9,1.1))是指從圖片中隨機切取出(224,224)的0.9\~1.1倍大的區域，縮放至(224,224)，因為它們的圖片超大也因此切到局部。  

另外，明暗調整前應該要加個AutoLevel，讓太暗的圖片可以變得比較清晰，也不至於暗的圖恰好遇到Gamma較大(變暗)的情形而變得更暗。此外，由於很多動物很小，所以在使用隨機擦除時，我將透明比例設為1，也就是不會有擦除部分不透明的情況(透明度介於0.4\~0.8之間)，胡椒鹽噪音用得更收斂些，只敢用到0.005。由於縮放是將整圖縮放至(224,224)，而且這個案例不會有不適用框間不變性的問題，因此我們額外再加入RandomTransformAffine。  

故意示範這個場景是要讓各位了解，不要把數據增強當作標準配方照抄，要按照目前數據情況做調整。


In [ ]:
data_provider.image_transform_funcs=[
    Resize((224,224)),
    AutoLevel(),
    RandomAdjustGamma(scale=(0.8,1.2)),#調整明暗
    RandomAdjustHue(scale=(-0.2,0.2)),#調整色相
    RandomAdjustSaturation(scale=(0.8,1.2)),#調整飽和度
    SaltPepperNoise(0.005, keep_prob=0.5),#加入胡椒鹽噪音
    RandomErasing(size_range=(0.05, 0.2), transparency_range=(0.4, 0.8), transparancy_ratio=1.0, keep_prob=0.5), #加入隨機擦去
    RandomTransformAffine(rotation_range=45, zoom_range=0.00, shift_range=0.00, shear_range=0.2, random_flip=0.15 ),#隨機仿射變換
    Normalize(127.5,127.5)] #標準化


In [ ]:
%%time
data_provider.preview_images()

In [ ]:
from trident.models import efficientnet

net1=efficientnet.EfficientNetB0(pretrained=True,include_top=True,classes=len(animal_category_lists_train),input_shape=(3,224,224),freeze_features=True)

net2=efficientnet.EfficientNetB0(pretrained=True,include_top=True,classes=len(animal_category_lists_train),input_shape=(3,224,224),freeze_features=True)

net1.summary()

這兩個baseline主要是用來確認，哪一個類別不平衡的校正策略比較好。請注意，由於建模圖片數量又多，圖檔尺寸又超大，即使只跑一個epoch都要很久，為了不浪費gpu時數資源，請注意，建議要把Resize類的轉換放在第一個，時間會差非常非常多...還有不要用原來的start_now()，改用only_steps(num_steps=步數)只需要指定步數即可。

In [ ]:
net1.with_optimizer(optimizer=AdaBelief,lr=1e-3,betas=(0.9, 0.999),gradient_centralization='all')\
.with_loss(CrossEntropyLoss(auto_balance=True))\
.with_metric(accuracy,name='accuracy')\
.with_metric(accuracy,topk=5,name='top5_accuracy')\
.with_regularizer('l2')\
.with_learning_rate_scheduler(reduce_lr_on_plateau,monitor='accuracy',mode='max',factor=0.5,patience=1,cooldown=0,threshold=5e-5,warmup=0)\
.with_model_save_path('Models/baseline_net1.pth')\
.with_automatic_mixed_precision_training()

net2.with_optimizer(optimizer=AdaBelief,lr=1e-3,betas=(0.9, 0.999),gradient_centralization='all')\
.with_loss(CrossEntropyLoss)\
.with_loss(FocalLoss,loss_weight=0.5)\
.with_metric(accuracy,name='accuracy')\
.with_metric(accuracy,topk=5,name='top5_accuracy')\
.with_regularizer('l2')\
.with_learning_rate_scheduler(reduce_lr_on_plateau,monitor='accuracy',mode='max',factor=0.5,patience=1,cooldown=0,threshold=5e-5,warmup=0)\
.with_model_save_path('Models/baseline_net2.pth')\
.with_automatic_mixed_precision_training()



plan=TrainingPlan()\
    .add_training_item(net1)\
    .add_training_item(net2)\
    .with_data_loader(data_provider)\
    .repeat_epochs(80)\
    .with_batch_size(256)\
    .print_gradients_scheduling(20,unit='batch') \
    .print_progress_scheduling(5,unit='batch') \
    .display_loss_metric_curve_scheduling(100)\
    .save_model_scheduling(10,unit='batch')

#因為圖片數量實在太大，所以改用only_steps
#而且圖片尺寸也很大，變成載入快取也要很久，建議把print_progress_scheduling設小一點
#第一次列印進度會較久(須完全載入快取)
plan.start_now()
